In [ ]:
import pandas as pd
import numpy as np
import re
import os

In [ ]:
carpeta_path = r'C:\Users\William\Desktop\el algoritmo es correcto'
archivo_train = r'train_data.csv'
path_train = os.path.join(carpeta_path, archivo_train)

In [ ]:
path_train

In [ ]:
train = pd.read_csv(
    path_train, 
    dtype={
        "tiempodeconstruido": str,
        "vista": str,
        "tipoinmueble": str,
        "tiponegocio": str,
        'valorventa': 'int64',
        # 'area': 'int64'
    }
)

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
for columna in train.columns:
    if columna not in ['id', 'valoradministracion', 'valorventa', 'latitud', 'longitud']:
        print("{}: ".format(columna), train[columna].unique())

In [ ]:
# Portería ==1, conjunto cerrado==0, tipoinmueble=='Casa'
train[
    (train['porteria'].isna()) &
    (train['conjuntocerrado']==0) &
    (train['tipoinmueble']=='Casa') &
    ((train['valoradministracion']>0) & (train['valoradministracion']<10000))
]


In [ ]:
train[
    (train['porteria'].isna()) &
    (train['conjuntocerrado']==0) &
    (train['tipoinmueble']=='Casa') &
    (train['ascensor'].isna())
]

In [ ]:
train[
    (train['porteria'].isna()) &
    (train['conjuntocerrado']==0) &
    (train['tipoinmueble']=='Casa') &
    # (train['valoradministracion'].isna() | train['valoradministracion']==0)
][['valoradministracion', 'estrato']].boxplot(by="estrato", figsize =(10,10))

In [ ]:
train[
    (train['gimnasio']==1) &
    (train['porteria'].isna()) &
    (train['conjuntocerrado']==0) &
    (train['tipoinmueble']=='Casa') &
    (train['valoradministracion'].isna() | train['valoradministracion']==0)
]['valoradministracion'].unique()

## Analizar columnas

### area

* file:///C:/Users/William/Downloads/406-Article%20Text-891-1-10-20181207%20(1).pdf

In [ ]:
train['area'].describe().astype('int64')

In [ ]:
train['area'].quantile(np.linspace(0,1,21)).astype('int64')

In [ ]:
train[train['area']<=100]['area'].plot(kind='box')

In [ ]:
train[
    (train['area']>=500) &
    (train['area']<=10000)]['area'].plot(kind='box')

In [ ]:
train[(train['area']>=1000)][['habitaciones', 'banos', 'estrato']].plot(kind = 'box')

In [ ]:
train[(train['area']>=500) & (train['estrato']==0)]

In [ ]:
# Propiedades con más de 20 años  es decir los que tengan más de 60 mts2
train[
    (train['tiempodeconstruido'].isin(['Entre 10 y 20 años','Más de 20 años','16 a 30 años'])) &
    (train['area']<60)
].shape

In [ ]:
# Propiedades con más de 30 años  es decir los que tengan más de 80 mts2
train[
    (train['tiempodeconstruido']=='Más de 30 años') &
    (train['area']<80)
].shape

In [ ]:
# Propiedades con menos de 20 años  es decir los que tengan más de 35 mts2
train[
    (train['tiempodeconstruido'].isin(
        ['Entre 10 y 20 años', 'Entre 0 y 5 años', 'Entre 5 y 10 años',
        '1 a 8 años', '9 a 15 años','Menos de 1 año','ntre 0 y 5 años'])) &
    (train['area']<35)
].shape

In [ ]:
# Propiedades que de entrada no cumplen los 35 mts
train[(train['area']<35)].shape

In [ ]:
train[(train['area']<35)][['id','area', 'habitaciones', 'banos', 'valorventa']]

In [ ]:
train.loc[10]

In [ ]:
train["y"] = train["valorventa"]/train["area"]

In [ ]:
# train["y"]
f = train[
    (train.valorventa < 800000000) & 
    (train.valorventa > 100000000) &
    (train.area < 5000) &
    (train.area > 35) &
    (train.habitaciones.notna()) &
    (train.banos.notna()) &
    (train.estrato.isin([4,6,7])) &
    (train.habitaciones.isin([1,3,8]))
]
f[["y","estrato", "habitaciones"]].boxplot(by=["estrato", "habitaciones"], figsize=(10,10))

In [ ]:
# id: 721798, 759440 Con estrato 0 son haciendas
# Zonas retiradas de bogotá

In [ ]:
import geopandas

In [ ]:
zipfile = r'C:\Users\William\Desktop\el algoritmo es correcto\comandooperativo.zip'
bogota = geopandas.read_file(zipfile, crs={'init': 'epsg:4326'})

In [ ]:
paraderositp_file = r'C:\Users\William\Desktop\el algoritmo es correcto\Paraderos_SITP_Bogota'
paraderositp = geopandas.read_file(paraderositp_file, crs={'init': 'epsg:4326'})

In [ ]:
carguedescargue_file = r'C:\Users\William\Desktop\el algoritmo es correcto\Zona_de_cargue_y_descargue'
carguedescargue = geopandas.read_file(carguedescargue_file, crs={'init': 'epsg:4326'})

In [ ]:
zonasamarillas_file = r'C:\Users\William\Desktop\el algoritmo es correcto\Zonas_Amarillas_Bogota_DC'
zonasamarillas = geopandas.read_file(zonasamarillas_file, crs={'init': 'epsg:4326'})

In [ ]:
semaforos_file = r'C:\Users\William\Desktop\el algoritmo es correcto\Red_Semaforica_de_Bogota_D_C'
semaforos = geopandas.read_file(semaforos_file, crs={'init': 'epsg:4326'})

In [ ]:
train_altovalor = train[
    (train['valorventa']>700000000) &
    (train['estrato']>5)
]

In [ ]:
altovalor = geopandas.GeoDataFrame(
    train_altovalor, geometry=geopandas.points_from_xy(train_altovalor.longitud, train_altovalor.latitud), crs={'init': 'epsg:4326'})

In [ ]:
altovalor

In [ ]:
altovalor.plot(color='green', figsize=(10, 10))
# estrato0 = estrato0.to_crs(bogota.crs)

In [ ]:
paraderositp.plot(color='blue', figsize=(10, 10))

In [ ]:
semaforos.plot(color='blue', figsize=(10, 10))

In [ ]:
altovalor = altovalor.to_crs(bogota.crs)
# paraderos_sitp = paraderositp.to_crs(estrato0.crs)
zonasamarillas = zonasamarillas.to_crs(bogota.crs)

In [ ]:
base1 = bogota.plot(color='white', edgecolor='black', figsize=(10, 10), zorder=1)
base2 = altovalor.plot(ax=base1, color='green', markersize=1, figsize=(10, 10), zorder=2)
semaforos.plot(ax=base2, color='red', markersize=1, figsize=(10, 10), zorder=3)

In [ ]:
bogota